# Image Classification Program 

#### Import all necessary packages before starting the program. This time usng Pytorch and Tf more.

In [4]:
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image

import tensorflow as tf
import torch

#import datasets and transform function
import torchvision 
import torchvision.transforms as transforms

import torch.nn as nn
import torch.optim as optim

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#### Load and Normalize Data

In [5]:
#define transform to convert images into Tensors then normalize them
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ]
)

#set batch size
batch_size = 4
#set number of workers
num_workers = 2

#load training data
trainset = torchvision.datasets.FashionMNIST(root='./data',train=True, download=True,
                                        transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, 
                                        shuffle=True, num_workers=num_workers)

#load test data
testset = torchvision.datasets.FashionMNIST(root='./data',train=False, download=True,
                                        transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=num_workers)


100%|██████████████████████████████| 26421880/26421880 [00:02<00:00, 10894323.73it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████████████████████████████████| 29515/29515 [00:00<00:00, 173627.07it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|█████████████████████████████████| 4422102/4422102 [00:01<00:00, 3418774.89it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|███████████████████████████████████████| 5148/5148 [00:00<00:00, 4761251.82it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



#### Set up a convolutional neural network model

In [6]:

class Net(nn.Module):
    
    def __init__(self):
        '''init the neural network'''
        super(Net, self).__init__()
        
        #set up 3x3 convolution kernels
        self.conv1 = nn.Conv2d(1, 6, 3)
        #set up 2x2 max pooling window
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, mod):
        '''defines the forward propogation algorithm'''
        #apply MaxPooling on the relu'd results from conv1/con2
        mod = self.pool(nn.functional.relu(self.conv1(mod)))
        mod = self.pool(nn.functional.relu(self.conv2(mod)))
        mod = mod.flatten(1)
        #run fc1 and fc2 layers through relu
        mod = nn.functional.relu(self.fc1(mod))
        mod = nn.functional.relu(self.fc2(mod))
        #fc3 will give us the final 10 layers
        mod = self.fc3(mod)
        return mod
    
net = Net()
print(net)
        
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### Define the loss func and optimzer

In [7]:
#similar to Keras's sparse categoricla cross-entropy 
crit = nn.CrossEntropyLoss()
#I used Adam in other program, just wanted to try sgd instead this time
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#### Train the nn

In [8]:
print(torch.cuda.is_available())

for epoch in range(5):
    r_loss = 0.0
    for i, data in enumerate(trainloader,0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = crit(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #print stats
        r_loss += loss.item()
        if i % 2000 == 1999: #print every 2000 iters
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, r_loss/2000))
            r_loss = 0.0


print('Finished Training')

False
[1,  2000] loss: 1.312
[1,  4000] loss: 0.683
[1,  6000] loss: 0.587
[1,  8000] loss: 0.526
[1, 10000] loss: 0.512
[1, 12000] loss: 0.485
[1, 14000] loss: 0.460
[2,  2000] loss: 0.427
[2,  4000] loss: 0.412
[2,  6000] loss: 0.401
[2,  8000] loss: 0.386
[2, 10000] loss: 0.392
[2, 12000] loss: 0.373
[2, 14000] loss: 0.367
[3,  2000] loss: 0.353
[3,  4000] loss: 0.361
[3,  6000] loss: 0.343
[3,  8000] loss: 0.343
[3, 10000] loss: 0.341
[3, 12000] loss: 0.324
[3, 14000] loss: 0.334
[4,  2000] loss: 0.314
[4,  4000] loss: 0.306
[4,  6000] loss: 0.311
[4,  8000] loss: 0.299
[4, 10000] loss: 0.304
[4, 12000] loss: 0.318
[4, 14000] loss: 0.318
[5,  2000] loss: 0.296
[5,  4000] loss: 0.277
[5,  6000] loss: 0.278
[5,  8000] loss: 0.290
[5, 10000] loss: 0.289
[5, 12000] loss: 0.293
[5, 14000] loss: 0.294
Finished Training


#### Run smaller dataset to check if anything is going horribly wrong

In [9]:
data_iter = iter(testloader)
images, labels = data_iter.next()

# print images

print('Actual: ', ' '.join('%d' % labels[j] for j in range(4)))

outputs = net(images)

_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%d' % predicted[j] for j in range(4)))

Actual:  9 2 1 1
Predicted:  9 2 1 1


#### Run net on test images dataset (10,000)

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy on 10000 test images: %.2f %%' % (100*correct/total))

Accuracy on 10000 test images: 87.05 %


#### Initial(Adam):
&emsp;Accuracy: 89.43% ---- 8943/10000
#### &#x21B3;Channel Size of Conv2D 16->32:
&emsp;Accuracy: 89.49% ---- 8949/10000
#### &#x21B3;Batch Size 4->8:
&emsp;Accuracy: 89.64% ---- 8964/10000
#### &#x21B3;LR .001->.0001:
&emsp;Accuracy: 86.76% ---- 8676/10000
#### &#x21B3;LR .001->.01:
&emsp;Accuracy: 77.98% ---- 7798/10000
#### Initial(SGD)
&emsp;Accuracy: 88.90% ---- 8890/10000
#### &#x21B3;LR .001->.0001:
&emsp;Accuracy: 83.64% ---- 8364/10000
#### &#x21B3;LR .001->.01:
&emsp;Accuracy: 78.96% ---- 7896/10000

In [ ]:
torch.save(net, '~/model.'